## Load Resume Dataset from Kaggle
https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset/data

In [26]:
# load data
import pandas as pd
df = pd.read_csv("Resume.csv")
print(df.head())

         ID  \
0  16852973   
1  22323967   
2  33176873   
3  27018550   
4  17812897   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Preprocessing 

In [27]:
df.shape

(2484, 4)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [29]:
df['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [30]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ADVOCATE                  118
CHEF                      118
ENGINEERING               118
ACCOUNTANT                118
FINANCE                   118
FITNESS                   117
AVIATION                  117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [31]:
# drop column 'Resume_html'
df = df.drop(columns=["Resume_html"])

In [32]:
# show full column width
pd.set_option('display.max_colwidth', None)
# show first row
df["Resume_str"].iloc[0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

In [33]:
import re

# Create a boolean column that marks resumes containing HTML tags
df["has_html"] = df["Resume_str"].apply(lambda x: bool(re.search(r"<.*?>", str(x))))

# Count how many resumes have HTML tags
print("Resumes with HTML tags:", df["has_html"].sum())
print("Resumes without HTML tags:", len(df) - df["has_html"].sum())



Resumes with HTML tags: 10
Resumes without HTML tags: 2474


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          2484 non-null   int64 
 1   Resume_str  2484 non-null   object
 2   Category    2484 non-null   object
 3   has_html    2484 non-null   bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 60.8+ KB


In [35]:
# Drop the 'has_html' column
df = df.drop(columns=['has_html'])

# readability
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r'<.*?>', ' ', text)               # remove HTML tags
    text = re.sub(r'\r', '', text)                   # remove returns
    text = re.sub(r'\n{3,}', '\n\n', text)           # limit multiple newlines to 2
    text = re.sub(r' {2,}', ' ', text)               # collapse extra spaces
    text = re.sub(r'(?<=\w)\n(?=\w)', ' ', text)     # avoid linebreaks mid-sentence
    
    # Add spacing before common section headers
    headers = [
        "summary", "highlights", "experience", "education",
        "projects", "skills", "certifications", "awards", "languages"
    ]
    for h in headers:
        pattern = rf"(?i)(?<=\b){h}\b"
        text = re.sub(pattern, f"\n\n{h.upper()}", text)
    
    text = text.strip()
    return text

# Apply the cleaning function to resumes
df["Resume_clean"] = df["Resume_str"].apply(clean_text)

# Quick check
df[["ID", "Category", "Resume_clean"]].head(3)


ID Category  \
0  16852973       HR   
1  22323967       HR   
2  33176873       HR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

### Entity Extraction (Use  in HR: Skill, Experience,Education etc.)

In [36]:
df.info()
df.to_csv("df.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            2484 non-null   int64 
 1   Resume_str    2484 non-null   object
 2   Category      2484 non-null   object
 3   Resume_clean  2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [37]:
# Filter for HR category only
df_hr = df[df["Category"].str.upper() == "HR"].reset_index(drop=True)

# Check how many HR resumes 
print("Number of HR resumes:", df_hr.shape[0])

# Quick preview
df_hr[["ID", "Category", "Resume_clean"]].head(3)



Number of HR resumes: 110


ID Category  \
0  16852973       HR   
1  22323967       HR   
2  33176873       HR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [38]:
df_hr.to_csv("hr_resumes.csv", index=False)

In [19]:
# # Load Ontology
# import json
# 
# with open("Ontology.json", "r", encoding="utf-8") as f:
#     ontology = json.load(f)
# 
# # Check top-level structure
# print("Ontology ID:", ontology.get("ontology_id"))
# print("Version:", ontology.get("version"))
# print("Entities:", list(ontology["entities"].keys()))


Ontology ID: cv_ontology_basic
Version: 1.0.0
Entities: ['education', 'work_experience', 'skills', 'language', 'certifications', 'projects', 'awards', 'voluntary experience', 'publications', 'hobbies']


In [20]:
# view
#print(json.dumps(ontology, indent=2))

{
  "ontology_id": "cv_ontology_basic",
  "version": "1.0.0",
  "description": "Ontology for CV extraction (knowledge recall)",
  "entities": {
    "education": {
      "description": "Formal degree or training",
      "attributes": {
        "degree_level": {
          "type": "enum",
          "values": [
            "bachelor",
            "master",
            "phd",
            "diploma",
            "certificate",
            "other"
          ]
        },
        "degree_title": {
          "type": "string"
        },
        "field": {
          "type": "string"
        },
        "year": {
          "type": "integer",
          "optional": true
        }
      },
      "evidence_required": true
    },
    "work_experience": {
      "description": "work experience",
      "attributes": {
        "job_title": {
          "type": "string"
        },
        "company": {
          "type": "string",
          "optional": true
        },
        "start_date": {
          "type": "st

In [24]:
# import os
# 
# # Create folder for text outputs
# os.makedirs("resume_texts", exist_ok=True)
# 
# # Randomly sample 20 HR resumes
# sample_df = df_hr.sample(n=20, random_state=42)
# 
# # Save each resume as a readable text file
# for _, row in sample_df.iterrows():
#     resume_id = row["ID"]
#     category = row["Category"]
#     text = row["Resume_clean"]
#     
#     # Add metadata header
#     formatted_text = (
#         f"ID: {resume_id}\n"
#         f"Category: {category}\n"
#         f"{'='*60}\n\n"
#         f"{text}"
#     )
#     
#     file_path = f"resume_texts/resume_{resume_id}.txt"
#     with open(file_path, "w", encoding="utf-8") as f:
#         f.write(formatted_text)


In [25]:
# import os
# # Folder for annotations
# with open("ontology.json", "r", encoding="utf-8") as f:
#     ontology = json.load(f)
# os.makedirs("annotations_manual", exist_ok=True)
# 
# # Function to build annotation template
# def make_annotation_json(resume_id, category, ontology):
#     def build_entity_template(entity_def):
#         attrs = {}
#         for attr in entity_def["attributes"].keys():
#             attrs[attr] = ""  # empty attributes
#         return [attrs]  # each entity starts as a list
# 
#     return {
#         "id": str(resume_id),
#         "category": category,
#         "ontology_id": ontology.get("ontology_id", ""),
#         "ontology_version": ontology.get("version", ""),
#         "entities": {
#             entity: build_entity_template(defn)
#             for entity, defn in ontology["entities"].items()
#         }
#     }
# 
# # Create and save a JSON file (sampled resume)
# for _, row in sample_df.iterrows():
#     resume_id = row["ID"]
#     category = row["Category"]
# 
#     annotation = make_annotation_json(resume_id, category, ontology)
# 
#     json_path = f"annotations_manual/annotated_{resume_id}.json"
#     with open(json_path, "w", encoding="utf-8") as f:
#         json.dump(annotation, f, indent=2, ensure_ascii=False)

In [ ]:
## prompt without pipeline

# 
# with open("Ontology.json", "r", encoding="utf-8") as f:  # load
#     ontology = json.load(f)
# 
# # folders
# resumes_dir = "resume_texts"
# output_dir = "model_output"
# os.makedirs(output_dir, exist_ok=True)

# #loop
# for file in os.listdir(resumes_dir):
#     if not file.endswith(".txt"):
#         continue
# 
#     resume_path = os.path.join(resumes_dir, file)
#     resume_id = os.path.splitext(file)[0].replace("resume_", "")
# 
#     with open(resume_path, "r", encoding="utf-8") as f:
#         resume_text = f.read()
# 
#     prompt = f"""
#     You are an HR specialist and information extraction expert.
# 
#     Your task:
#     Given a candidate’s resume, extract key information according to the following ontology.
#     Follow these rules:
#     - Use only what is written in the resume.
#     - Keep responses factual and concise.
#     - If information is missing, leave the field empty.
#     - If multiple values appear for a field, include all of them as a list.
#     - Output valid JSON only. (no extra text, explanations, or formatting outside JSON).
# 
#     Ontology schema to follow:
#     {json.dumps(ontology, indent=2)}
# 
#     Resume text:
#     {resume_text}
#     """
#  # call llm, add meta data, save output in json in folder
#     if use_proprietary_llms:
#         if api_keys_exist(env_vars=llm_env_vars, debug=True):
#             result = ask_llm(prompt=prompt, model=gemini_model)
#             clean_result = result.strip()
#             if clean_result.startswith("```"):
#                 clean_result = clean_result.strip("`")
#                 clean_result = clean_result.replace("json", "", 1).strip()
# 
#             if clean_result:
#                 parsed = json.loads(clean_result)
#                 parsed["id"] = resume_id
#                 parsed["category"] = "HR"
#                 parsed["ontology_id"] = ontology.get("ontology_id", "")
#                 parsed["ontology_version"] = ontology.get("version", "")
# 
#                 output_path = os.path.join(output_dir, f"model_output_{resume_id}.json")
#                 # add meta data
#                 ordered = {
#                     "id": parsed.get("id", ""),
#                     "category": parsed.get("category", ""),
#                     "ontology_id": parsed.get("ontology_id", ""),
#                     "ontology_version": parsed.get("ontology_version", "")
#                 }
# 
#                 for key, value in parsed.items():
#                     if key not in ordered:
#                         ordered[key] = value
# 
#                 # Save ordered JSON
#                 with open(output_path, "w", encoding="utf-8") as f:
#                     json.dump(ordered, f, indent=2, ensure_ascii=False)





